# Inference with `google/gemma-7b-it`. RAG experiment

## Purpose
This notebook performs inference using the `google/gemma-7b-it` language model from Hugging Face's `transformers` library. The test dataset consists of 1363 records, which are fetched via an HTTP call to the **Palaven API**—a custom API developed exclusively for this thesis. The generated responses, along with evaluation metrics, are saved back to the Palaven API for future analysis.

## Overview

1. **Declaration of Global Variables**:
    - The notebook initializes all necessary global variables, including paths, API credentials, batch sizes, and any configuration settings required for inference and evaluation.

2. **Loading the Model and Tokenizer**:
    - The `google/gemma-7b-it` model and its corresponding tokenizer are loaded from the Hugging Face library (`transformers`).
    - The model is used in its "out of the box" configuration, meaning no additional fine-tuning is applied at this stage.
    - **Retrieval-Augmented Generation (RAG)** is incorporated, leveraging external knowledge from a retrieval system, enabling the model to generate more informed responses.

3. **Inference Execution**:
    - **Dataset fetching**: The test dataset, consisting of 1363 records, is retrieved from the cloud via the **Palaven API** using an HTTP request.
    - **Batch processing**: The dataset is processed in batches, with each batch containing 50 instructions. This is done to optimize memory usage and ensure smooth processing during inference.
    - **Model Inference**: For each instruction in the dataset, the `google/gemma-7b-it` model generates responses, which are enhanced by the RAG component.
    - **Saving responses**: All generated responses are persisted back to the Palaven API for future evaluation and analysis.

4. **Evaluation Metrics**:
    - **BERTScore**: Precision, Recall, and F1 scores are calculated for each batch. This metric is particularly useful for evaluating the semantic similarity between the generated text and the ground truth.
    - **ROUGE**: ROUGE 1, ROUGE 2, and ROUGE L are computed to measure n-gram overlap between the generated responses and the ground truth. Precision, Recall, and F1 are evaluated for each.
    - **BLEU**: The BLEU score is calculated for each batch to assess the fluency and correctness of the generated text based on n-gram precision.
    - **Persisting metrics**: All computed metrics are persisted to the cloud via the Palaven API for deeper analysis and comparison across experiments.

## Key Sections
### 1. Declaration of global variables

| **Variable**             | **Description**                                                                                   | **Value**                                                   |
|--------------------------|---------------------------------------------------------------------------------------------------|-------------------------------------------------------------|
| `hface_read_token`        | Access token for reading from Hugging Face, retrieved from user data.                             | `userdata.get('hface-read-token')`                          |
| `palaven_base_url`        | Base URL for the Palaven API, retrieved from user data.                                           | `userdata.get('palaven-base-url')`                          |
| `batch_size`              | Batch size for inference and evaluation execution.                                                | `50`                                                        |
| `dataset_id`              | Unique identifier for the dataset used in evaluation.                                             | `'F0444B12-5485-4299-B03B-3BDB6D4A2578'`                    |
| `evaluation_session_id`   | Unique identifier for the current evaluation session.                                             | `'EB9C5839-7B20-4D7D-B3F7-17528180676D'`                    |
| `llm_palaven_name`        | Name of the model used in Palaven.                                                 | `'google-gemma'`                                            |
| `llm_model_name`          | Full name of the model used from Hugging Face.                                                    | `'google/gemma-7b-it'`                                      |
| `device_info`             | Information about the device where the inferences and evaluations are executed.                   | `'GPU A100'`                                                |


In [4]:
from google.colab import userdata
from palaven_api_v2 import PalavenApi

hface_read_token = userdata.get('hface-read-token')
palaven_base_url = userdata.get('palaven-base-url')

batch_size = 50
dataset_id = 'F0444B12-5485-4299-B03B-3BDB6D4A2578'
evaluation_session_id = 'EB9C5839-7B20-4D7D-B3F7-17528180676D'
llm_palaven_name = 'google-gemma'
llm_model_name = 'google/gemma-7b-it'
device_info = 'GPU A100'

palaven_api = PalavenApi(palaven_base_url)

### 2. Load the model and the tokenizer

In [ ]:
!pip install -U transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 83.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(llm_model_name, token=hface_read_token)
model = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map="auto", token=hface_read_token).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### 3. Inference Execution

In [ ]:
import time
import torch

def get_chat_completion(instruction_prompt, instruction_id, instruction_count):

  start_time = time.time()

  tokenized_instruction = tokenizer(instruction_prompt, return_tensors="pt").to("cuda")

  with torch.no_grad():
    chat_completion_result = model.generate(**tokenized_instruction, max_new_tokens=1000)

  chat_completion = tokenizer.decode(chat_completion_result[0])

  end_time = time.time()
  elapsed_time = end_time - start_time

  print(f'{instruction_count} - LLM-ChatCompletion. InstructionId: {instruction_id},  Elapsed-Time: {elapsed_time:.2f} seconds')

  return chat_completion, elapsed_time

In [ ]:
def add_instruction_to_df(df, instruction_id, instruction):
    df.loc[instruction_id, 'instruction'] = instruction
    df.loc[instruction_id, 'chat_completion'] = None
    df.loc[instruction_id, 'elapsed_time'] = None

In [ ]:
import pandas as pd
import numpy as np
import json

data_shape = {
    'instruction_id': [],
    'instruction': [],
    'chat_completion': [],
    'elapsed_time': []
}

for batch_number in range(26, 29):

  print(f'Start fetching batch {batch_number}')

  instructions = palaven_api.fetch_instruction_test_dataset(evaluation_session_id, batch_number=batch_number, evaluation_exercise='llmrag')

  print(f'Start fetching batch done...')

  llm_responses_df = pd.DataFrame(data_shape)
  llm_responses_df.set_index('instruction_id', inplace=True)

  for item in instructions:
    add_instruction_to_df(llm_responses_df, item['instructionId'], item['instruction'])

  instruction_count = 1

  for index, row in llm_responses_df.iterrows():
    instruction = llm_responses_df.loc[index, 'instruction']

    query_augmentation_response = palaven_api.generate_augmented_prompt(llm_palaven_name, instruction)
    instruction_prompt = query_augmentation_response['prompt']

    instruction_id = int(index)

    chat_completion, elapsed_time = get_chat_completion(instruction_prompt, instruction_id, instruction_count)
    palaven_api.save_model_response(evaluation_session_id, 'llmrag', batch_number, instruction_id, chat_completion, elapsed_time)

    torch.cuda.empty_cache()

    instruction_count += 1

Start fetching batch 26
Start fetching batch done...


<ipython-input-6-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Las personas morales que distribuyan dividendos o utilidades a residentes en México deben retener un impuesto del 10% sobre dichos pagos?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 7463,  Elapsed-Time: 5.49 seconds
Palaven.SaveResponse. InstructionId: 7463,  Elapsed-Time: 0.52 seconds
2 - LLM-ChatCompletion. InstructionId: 7465,  Elapsed-Time: 6.61 seconds
Palaven.SaveResponse. InstructionId: 7465,  Elapsed-Time: 0.20 seconds
3 - LLM-ChatCompletion. InstructionId: 7474,  Elapsed-Time: 17.77 seconds
Palaven.SaveResponse. InstructionId: 7474,  Elapsed-Time: 0.24 seconds
4 - LLM-ChatCompletion. InstructionId: 7476,  Elapsed-Time: 3.83 seconds
Palaven.SaveResponse. InstructionId: 7476,  Elapsed-Time: 0.18 seconds
5 - LLM-ChatCompletion. InstructionId: 7478,  Elapsed-Time: 1.45 seconds
Palaven.SaveResponse. InstructionId: 7478,  Elapsed-Time: 0.17 seconds
6 - LLM-ChatCompletion. InstructionId: 7480,  Elapsed-Time: 1.94 seconds
Palaven.SaveResponse. InstructionId: 7480,  Elapsed-Time: 0.23 seconds
7 - LLM-ChatCompletion. InstructionId: 7482,  Elapsed-Time: 2.26 seconds
Palaven.SaveResponse. InstructionId: 7482,  Elapsed-Time: 0.20

<ipython-input-6-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Sumariza el artículo 169 de la Ley del Impuesto Sobre la Renta de 2024' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 7577,  Elapsed-Time: 8.45 seconds
Palaven.SaveResponse. InstructionId: 7577,  Elapsed-Time: 0.18 seconds
2 - LLM-ChatCompletion. InstructionId: 7579,  Elapsed-Time: 4.43 seconds
Palaven.SaveResponse. InstructionId: 7579,  Elapsed-Time: 0.23 seconds
3 - LLM-ChatCompletion. InstructionId: 7581,  Elapsed-Time: 6.89 seconds
Palaven.SaveResponse. InstructionId: 7581,  Elapsed-Time: 0.19 seconds
4 - LLM-ChatCompletion. InstructionId: 7583,  Elapsed-Time: 1.88 seconds
Palaven.SaveResponse. InstructionId: 7583,  Elapsed-Time: 0.21 seconds
5 - LLM-ChatCompletion. InstructionId: 7585,  Elapsed-Time: 6.44 seconds
Palaven.SaveResponse. InstructionId: 7585,  Elapsed-Time: 0.18 seconds
6 - LLM-ChatCompletion. InstructionId: 7587,  Elapsed-Time: 2.27 seconds
Palaven.SaveResponse. InstructionId: 7587,  Elapsed-Time: 0.22 seconds
7 - LLM-ChatCompletion. InstructionId: 7589,  Elapsed-Time: 4.84 seconds
Palaven.SaveResponse. InstructionId: 7589,  Elapsed-Time: 0.21 

<ipython-input-6-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Los residentes en México y los residentes en el extranjero con establecimiento permanente en el país están obligados a pagar impuestos por los ingresos sujetos a regímenes fiscales preferentes obtenidos a través de entidades extranjeras en las que participen?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 7712,  Elapsed-Time: 9.49 seconds
Palaven.SaveResponse. InstructionId: 7712,  Elapsed-Time: 0.22 seconds
2 - LLM-ChatCompletion. InstructionId: 7714,  Elapsed-Time: 9.57 seconds
Palaven.SaveResponse. InstructionId: 7714,  Elapsed-Time: 0.26 seconds
3 - LLM-ChatCompletion. InstructionId: 7716,  Elapsed-Time: 13.15 seconds
Palaven.SaveResponse. InstructionId: 7716,  Elapsed-Time: 0.23 seconds
4 - LLM-ChatCompletion. InstructionId: 7725,  Elapsed-Time: 25.67 seconds
Palaven.SaveResponse. InstructionId: 7725,  Elapsed-Time: 0.26 seconds
5 - LLM-ChatCompletion. InstructionId: 7727,  Elapsed-Time: 10.50 seconds
Palaven.SaveResponse. InstructionId: 7727,  Elapsed-Time: 0.24 seconds
6 - LLM-ChatCompletion. InstructionId: 7729,  Elapsed-Time: 19.27 seconds
Palaven.SaveResponse. InstructionId: 7729,  Elapsed-Time: 0.27 seconds
7 - LLM-ChatCompletion. InstructionId: 7731,  Elapsed-Time: 10.14 seconds
Palaven.SaveResponse. InstructionId: 7731,  Elapsed-Time: 

### 4. Evaluation metrics

### 4.1. BERTScore

In [3]:
import pandas as pd

def build_responses_df():
  dataframe = pd.DataFrame(columns=['instruction_id',
    'evaluation_session_id', 'dataset_id', 'batch_size', 'large_language_model',
    'device_info', 'excercise_type', 'batch_number', 'instruction',
    'response', 'category', 'response_to_evaluate', 'elapsed_time'])

  dataframe['evaluation_session_id'] = dataframe['evaluation_session_id'].astype('object')
  dataframe['dataset_id'] = dataframe['dataset_id'].astype('object')
  dataframe['large_language_model'] = dataframe['large_language_model'].astype('object')
  dataframe['device_info'] = dataframe['device_info'].astype('object')
  dataframe['excercise_type'] = dataframe['excercise_type'].astype('object')
  dataframe['instruction'] = dataframe['instruction'].astype('object')
  dataframe['response'] = dataframe['response'].astype('object')
  dataframe['category'] = dataframe['category'].astype('object')
  dataframe['response_to_evaluate'] = dataframe['response_to_evaluate'].astype('object')

  dataframe.set_index('instruction_id', inplace=True)

  return dataframe


def add_response_to_df(dataframe, response):
  dataframe.loc[response['instructionId'], 'evaluation_session_id'] = response['evaluationSessionId']
  dataframe.loc[response['instructionId'], 'dataset_id'] = response['datasetId']
  dataframe.loc[response['instructionId'], 'batch_size'] = response['batchSize']
  dataframe.loc[response['instructionId'], 'large_language_model'] = response['largeLanguageModel']
  dataframe.loc[response['instructionId'], 'device_info'] = response['deviceInfo']
  dataframe.loc[response['instructionId'], 'excercise_type'] = response['evaluationExercise']
  dataframe.loc[response['instructionId'], 'batch_number'] = response['batchNumber']
  dataframe.loc[response['instructionId'], 'instruction'] = response['instruction']
  dataframe.loc[response['instructionId'], 'response'] = response['response']
  dataframe.loc[response['instructionId'], 'category'] = response['category']
  dataframe.loc[response['instructionId'], 'response_to_evaluate'] = response['llmResponseToEvaluate']
  dataframe.loc[response['instructionId'], 'elapsed_time'] = response['elapsedTime']

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
from bert_score import score as bert_score

for batch_number in range(1, 29):

  model_responses_df = build_responses_df()

  print(f'Palaven. Bertscore evaluation. Start processing batch: {batch_number}')

  model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmrag', batch_number)

  for item in model_responses:
    add_response_to_df(model_responses_df, item)

  references = model_responses_df['response'].tolist()
  candidates = model_responses_df['response_to_evaluate'].tolist()

  accuracy, recall, f1 = bert_score(candidates, references, lang='es', verbose=True, device='cuda')

  accuracy = accuracy.tolist()
  recall = recall.tolist()
  f1 = f1.tolist()

  average_accuracy = sum(accuracy) / len(accuracy)
  average_recall = sum(recall) / len(recall)
  average_f1 = sum(f1) / len(f1)

  print(f'Palaven. Bertscore evaluation. Batch: {batch_number}. Average accuracy: {average_accuracy}. Average recall: {average_recall}. Average F1: {average_f1}')

  palaven_api.save_bert_score_metrics(evaluation_session_id, 'llmrag', batch_number, average_accuracy, average_recall, average_f1)

  print(f'Palaven. Bertscore evaluation. Posted batch {batch_number}')

Palaven. Bertscore evaluation. Start processing batch: 1
calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.66 seconds, 76.32 sentences/sec
Palaven. Bertscore evaluation. Batch: 1. Average accuracy: 0.6811982405185699. Average recall: 0.7907091915607453. Average F1: 0.7308437764644623
Palaven. Bertscore evaluation. Posted batch 1
Palaven. Bertscore evaluation. Start processing batch: 2
calculating scores...
computing bert embedding.


...

done in 0.63 seconds, 79.52 sentences/sec
Palaven. Bertscore evaluation. Batch: 27. Average accuracy: 0.7071169996261597. Average recall: 0.8157383966445922. Average F1: 0.7565126657485962
Palaven. Bertscore evaluation. Posted batch 27
Palaven. Bertscore evaluation. Start processing batch: 28
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.25 seconds, 51.28 sentences/sec
Palaven. Bertscore evaluation. Batch: 28. Average accuracy: 0.7452399272185105. Average recall: 0.8599351002619817. Average F1: 0.7978805670371423
Palaven. Bertscore evaluation. Posted batch 28


### 4.2. ROUGE (1,2 and L)

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=27ffbc8d5e06254b5fcce463fdfb02cba244231f7c533e7317244ab18edaaae2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

for batch_number in range(1, 29):

  model_responses_df = build_responses_df()

  print(f'Palaven. ROUGE evaluation. Start processing batch: {batch_number}')
  model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmrag', batch_number)

  for item in model_responses:
    add_response_to_df(model_responses_df, item)

  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

  r1_precision, r1_recall, r1_f1 = [], [], []
  r2_precision, r2_recall, r2_f1 = [], [], []
  rL_precision, rL_recall, rL_f1 = [], [], []

  for _, row in model_responses_df.iterrows():
    reference = row['response']
    candidate = row['response_to_evaluate']

    scores = scorer.score(reference, candidate)

    #ROUGE-1
    r1_precision.append(scores['rouge1'].precision)
    r1_recall.append(scores['rouge1'].recall)
    r1_f1.append(scores['rouge1'].fmeasure)

    #ROUGE-2
    r2_precision.append(scores['rouge2'].precision)
    r2_recall.append(scores['rouge2'].recall)
    r2_f1.append(scores['rouge2'].fmeasure)

    #ROUGE-L
    rL_precision.append(scores['rougeL'].precision)
    rL_recall.append(scores['rougeL'].recall)
    rL_f1.append(scores['rougeL'].fmeasure)


  average_r1_precision = sum(r1_precision) / len(r1_precision)
  average_r1_recall = sum(r1_recall) / len(r1_recall)
  average_r1_f1 = sum(r1_f1) / len(r1_f1)

  average_r2_precision = sum(r2_precision) / len(r2_precision)
  average_r2_recall = sum(r2_recall) / len(r2_recall)
  average_r2_f1 = sum(r2_f1) / len(r2_f1)

  average_rL_precision = sum(rL_precision) / len(rL_precision)
  average_rL_recall = sum(rL_recall) / len(rL_recall)
  average_rL_f1 = sum(rL_f1) / len(rL_f1)

  rouge_metrics = []

  rouge_metrics.append({
      'rougeScoreType': 'rouge1',
      'batchNumber': batch_number,
      'precision': average_r1_precision,
      'recall': average_r1_recall,
      'f1': average_r1_f1
  });

  rouge_metrics.append({
      'rougeScoreType': 'rouge2',
      'batchNumber': batch_number,
      'precision': average_r2_precision,
      'recall': average_r2_recall,
      'f1': average_r2_f1
  });

  rouge_metrics.append({
      'rougeScoreType': 'rougeL',
      'batchNumber': batch_number,
      'precision': average_rL_precision,
      'recall': average_rL_recall,
      'f1': average_rL_f1
  });

  print(rouge_metrics)

  print(f'Palaven. ROUGE evaluation finished for batch: {batch_number}.')

  palaven_api.save_rouge_metrics(evaluation_session_id, 'llmrag', batch_number, rouge_metrics)

  print(f'Palaven. Bertscore evaluation. Posted batch {batch_number}')

Palaven. ROUGE evaluation. Start processing batch: 1
[{'rougeScoreType': 'rouge1', 'batchNumber': 1, 'precision': 0.24074263710291113, 'recall': 0.783343800546673, 'f1': 0.34376082035980504}, {'rougeScoreType': 'rouge2', 'batchNumber': 1, 'precision': 0.16760326312177753, 'recall': 0.5030134097091734, 'f1': 0.23588160727412824}, {'rougeScoreType': 'rougeL', 'batchNumber': 1, 'precision': 0.20583240074308126, 'recall': 0.6674728113140467, 'f1': 0.2924526596197256}]
Palaven. ROUGE evaluation finished for batch: 1.
Palaven. Bertscore evaluation. Posted batch 1
Palaven. ROUGE evaluation. Start processing batch: 2
[{'rougeScoreType': 'rouge1', 'batchNumber': 2, 'precision': 0.3858040655561248, 'recall': 0.8418402253840505, 'f1': 0.49852663184229795}, {'rougeScoreType': 'rouge2', 'batchNumber': 2, 'precision': 0.31109939158782124, 'recall': 0.6803819627630583, 'f1': 0.40279677357866}, {'rougeScoreType': 'rougeL', 'batchNumber': 2, 'precision': 0.34392155020998627, 'recall': 0.751895604049797

### 4.3. BLEU

In [5]:
from nltk.translate.bleu_score import sentence_bleu

for batch_number in range(1, 29):

    model_responses_df = build_responses_df()

    print(f'Palaven. BLEU evaluation. Start processing batch: {batch_number}')

    model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmrag', batch_number)

    for item in model_responses:
        add_response_to_df(model_responses_df, item)

    references = model_responses_df['response'].tolist()
    candidates = model_responses_df['response_to_evaluate'].tolist()

    bleu_scores = []

    for candidate, reference in zip(candidates, references):
        reference_tokens = [reference.split()]
        candidate_tokens = candidate.split()
        bleu_score = sentence_bleu(reference_tokens, candidate_tokens)
        bleu_scores.append(bleu_score)

    average_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f'Palaven. BLEU evaluation. Batch: {batch_number}. Average BLEU score: {average_bleu}')

    palaven_api.save_bleu_score_metrics(evaluation_session_id, 'llmrag', batch_number, average_bleu)

    print(f'Palaven. BLEU evaluation. Posted batch {batch_number}')

Palaven. BLEU evaluation. Start processing batch: 1


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Palaven. BLEU evaluation. Batch: 1. Average BLEU score: 0.12193979289146518
Palaven. BLEU evaluation. Posted batch 1
Palaven. BLEU evaluation. Start processing batch: 2


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Palaven. BLEU evaluation. Batch: 2. Average BLEU score: 0.2583534253422665
Palaven. BLEU evaluation. Posted batch 2
Palaven. BLEU evaluation. Start processing batch: 3


...

Palaven. BLEU evaluation. Batch: 25. Average BLEU score: 0.15861313726851467
Palaven. BLEU evaluation. Posted batch 25
Palaven. BLEU evaluation. Start processing batch: 26


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 26. Average BLEU score: 0.13180036892983
Palaven. BLEU evaluation. Posted batch 26
Palaven. BLEU evaluation. Start processing batch: 27


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Palaven. BLEU evaluation. Batch: 27. Average BLEU score: 0.14851958144557492
Palaven. BLEU evaluation. Posted batch 27
Palaven. BLEU evaluation. Start processing batch: 28
Palaven. BLEU evaluation. Batch: 28. Average BLEU score: 0.21869266794146613
Palaven. BLEU evaluation. Posted batch 28
